# Map Making With Python #


## Introduction ##
![Heatmap of Restaurants in Boston](heatmap.png)
Have you ever wondered how people make cool maps with data? This notebook will walk you through the steps for programmatically generating a heatmap and a choropleth map using just a few python packages. By the end, you'll know basic, transferable skills for making maps and be ready to do more on your own.  

## Python Basics ##  

Some of the most basic types in python are strings, integers, floats, and booleans. Variables can be any of these and more.  

In [ ]:
#Comments can follow '#' or be enclosed with triple quotes """ """

x = 2 #This is an int
print(x)
print(type(x))

x = 'this is a string' #strings can be enclosed with single or double quotes 'hello' "hi"
print(x)
print(type(x))

#x = this doesnt work

In [ ]:
#Using operators

#On ints
print(3 + 2)
x = 3
y = 2
print(x + y)

#On strings
print('hello' + 'there')
x = 'hello there' * 3
print(x)

In [ ]:
#Using functions

def add(x, y):    #We define a function called add with the parameters x and y
    print(x + y)  #It prints out there sum

add(2,3) #Add the arguments 2 and 3
add('hello', 'there')
add(True, False)

#in jupyter, if you ever want information about a function, 
#you can type it's name followed by ?
print?

In [ ]:
#Objects

#simple types like int and string are objects. An object is a bundle of some data,
#and functions that act on that data.
#These functions within objects are called

#methods
x = 'this is all lowercase'
print(x)
print(x.upper())

In [ ]:
#Collections 

print('Lists:')
#Lists: ordered, mutable, indexable, versatile and great for storing elements
L = [1,2,3,4]
print(L)
print(L[0])

print('\nDictionaries:')
#Dictionaries: Key-Value pairs
name = 'brian'
D = {'key1' : 'value1', 'Name' : name}
print(D)
print(D['Name'])

print('\nfun:')
#Values in lists and dicts can be more than just strings and ints
weatherdata = {'Town' : ['Boston', 'Salem', 'Worcester', 'Ludlow']
              ,'Temp' : [65, 63, 67, 58]
              ,'Forecast': ['Rain', 'Rain', 'Cloudy', 'Sunny']}

print(weatherdata)

print('\nPandas Dataframe')
#From pandas
#DataFrames
import pandas as pd
df = pd.DataFrame(weatherdata)
print(df)
#pd.DataFrame?

## Let's Make a Map! ##

[Folium](https://python-visualization.github.io/folium/modules.html) - easy to use and great for creating interactive leaflet maps  
[JSON](https://docs.python.org/3/library/json.html) - JavaScript Object Notation encoder/decoder for reading in geographical data  
[pandas](https://pandas.pydata.org/pandas-docs/stable/reference/index.html) - data structures for easy manipulation and analysis  
  
Data was taken from **https://data.boston.gov/** and **https://data.worldbank.org/indicator/it.NET.user.ZS**  



In [ ]:
import folium
from folium.plugins import HeatMap
import json
import pandas as pd

In [ ]:
#Let's see what the data looks like
pd.read_csv('restaurants.csv')

In [ ]:
#Now load it into a variable and get the data we need
restaurants = pd.read_csv('restaurants.csv', usecols = ['BusinessName','Latitude','Longitude']) #Read in only data for these columns
restaurants = restaurants.drop_duplicates()
restaurants['Latitude'].apply(int) #make sure the value is int
restaurants['Longitude'].apply(int)
print(restaurants)

#remove entries without coordinates
restaurants = restaurants[restaurants.Latitude != 0]
print('Prepared data:\n', restaurants)

In [ ]:
#Read in neighborhood boundaries
with open('Boston_Neighborhoods.geojson', 'r') as file:
    Neighborhoods = json.load(file)

In [ ]:
#Now let's make the map!
map = folium.Map(location = [42.3601, -71.0589] ,zoom_start = 14)#set the basemap starting location and zoom

heatmap = HeatMap(list(zip(restaurants['Latitude'], restaurants['Longitude']))#creating a separate heatmap
                  ,min_opacity=0.2
                  ,max_val=1
                  ,radius=10, blur=15
                  ,max_zoom=1
                 )
heatmap.caption= 'Restaurants in Boston with Available Coordinates'

In [ ]:
#add the neighborhood boundaries
folium.GeoJson(Neighborhoods).add_to(map)

In [ ]:
#add the heatmap
map.add_child(heatmap)

In [ ]:
#Now let's make another map!
#First load in the data
internet = pd.read_csv('Internet.csv', skiprows=4).dropna(axis=1, how='all')

with open('country-codes.json', 'r') as file:
    countrycodes = json.load(file)



In [ ]:
#Let's take a peak at the data
internet.head()

In [ ]:
print(countrycodes)

In [ ]:
#Now let's select the data we'd like to plot
codes = [i['id'] for i in countrycodes['features']]
datacodes = internet['Country Code']
#for i in datacodes:
#    if i not in codes:
#        print(i)

internet = internet[internet['Country Code'].isin(codes)]

internet2000 = internet[['Country Code', '2000']]
internet2010 = internet[['Country Code', '2010']]
internet2017 = internet[['Country Code', '2017']]
internet2005 = internet[['Country Code', '2005']]

In [ ]:
print(internet2000)

In [ ]:
m1 = folium.Map(location = [25,0], zoom_start = 1.5)
m2 = folium.Map(location = [25,0], zoom_start = 1.5)
m3 = folium.Map(location = [25,0], zoom_start = 1.5)
bins = [float(num) for num in range(0,110,10)]      

folium.Choropleth(geo_data = countrycodes
            ,data = internet2017
            ,columns = ['Country Code', '2017']
            ,key_on = 'feature.id'
            ,fill_color = 'Spectral'
            ,fill_opacity = 0.7
            ,line_opacity = 0.4
            ,legend_name = 'People with Internet Access in 2017 (%)'
            ,name = '2017'
            ,bins = bins
            ).add_to(m1)

folium.Choropleth(geo_data = countrycodes
            ,data = internet2010
            ,columns = ['Country Code', '2010']
            ,key_on = 'feature.id'
            ,fill_color = 'Spectral'
            ,fill_opacity = 0.7
            ,line_opacity = 0.4
            ,legend_name = 'People with Internet Access in 2010 (%)'
            ,name = '2010'
            ,bins = bins
            ).add_to(m2)

folium.Choropleth(geo_data = countrycodes
            ,data = internet2005
            ,columns = ['Country Code', '2005']
            ,key_on = 'feature.id'
            ,fill_color = 'Spectral'
            ,fill_opacity = 0.7
            ,line_opacity = 0.4
            ,legend_name = 'People with Internet Access in 2005 (%)'
            ,name = '2005'
            ,bins = bins
            ).add_to(m3)

#folium.LayerControl().add_to(m)

In [ ]:
m1


In [ ]:
m2

In [ ]:
m3